<a href="https://colab.research.google.com/github/ideablast/NLPer_chatbot/blob/hj/FAQ_doc2vec_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.4MB/s 
     |████████████████████████████████| 460kB 35.7MB/s 
     |████████████████████████████████| 92kB 7.9MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from gensim.models import doc2vec
from gensim.models.doc2vec import TaggedDocument
import pandas as pd

In [ ]:
wear_data = pd.read_csv("/content/drive/My Drive/clothing.csv")
print(wear_data.shape)
customer = wear_data[wear_data.SPEAKER == "고객"].SENTENCE
store = wear_data[wear_data.SPEAKER == "점원"].SENTENCE
print(customer.shape, store.shape) # 질문의 개수와 답의 개수가 일치하지 않는다.
store.head()
print(type(wear_data))

(15826, 20)
(8381,) (7445,)
<class 'pandas.core.frame.DataFrame'>


In [ ]:
prev = "고객"
store_arr = []
customer_arr = []
store_stc = ""
customer_stc = ""

for i in range(wear_data.shape[0]):
    if (prev == wear_data.iloc[i].SPEAKER):
        if prev == "점원":
             store_stc += (" "+wear_data.iloc[i].SENTENCE)
        else : 
            customer_stc += (" "+wear_data.iloc[i].SENTENCE)
            
    elif prev == "점원":
        store_arr.append(store_stc)
        customer_stc = wear_data.iloc[i].SENTENCE
        prev = "고객"
    else :
        customer_arr.append(customer_stc)
        store_stc = wear_data.iloc[i].SENTENCE
        prev = "점원"

print(len(store_arr))
print(len(customer_arr))

faqs = []

for i in range(len(store_arr)):
    faqs_tmp =[]
    faqs_tmp.append(str(i+1))
    faqs_tmp.append(customer_arr[i])
    faqs_tmp.append(store_arr[i])

    faqs.append(faqs_tmp)

# print(store_arr[-1])
# print(customer_arr[-1]) # 자료 상에서 이후에는 계속 고객의 물음만 계속된다. 코드 레벨에서 이 부분은 빼게 구현했다. (stc는 만들어지지만 arr에 append 안하게 된다.)
faqs

7301
7301


[['1', ' 신발은 여기 있는 게 다예요?', '네 성인이나 아동 다 있어요 발 사이즈 몇 신으세요?'],
 ['2', '230이요', '편하게 신을 수 있는 거 찾으세요?'],
 ['3', '네 봄이니까 편하게 신을 수 있는 거', '이런 건 어떠세요? 이런 거도 신발 무척 편하거든요'],
 ['4', '굽 좀 높은 거 없나요?', '봄 상품은 아직 어른 제품이 많이 안나왔습니다'],
 ['5', '언제 들어와요?', '이번주 지나면 들어올 거예요'],
 ['6', '이거는 가죽이에요?', '가죽 아니고 쎄무예요'],
 ['7', '가죽은 얼마예요?', '2만 9천 원입니다'],
 ['8', '털 달린 거 저거는 사이즈 있어요?', '230이 없어요  이거 한 번 신어보세요'],
 ['9', '좀 크네 또 안 들어와요?', '네 이건 다 끝났어요'],
 ['10', '가방 매는 거 보고 있어요', '여기 있어요'],
 ['11', '가격이 얼마예요?', '이 종류는 2만 원이고 이 종류는 3만 8천 원이에요'],
 ['12', '가죽으로 된 거는 없어요?', '가죽은 없고 레자만 있어요'],
 ['13', '레자는 얼마예요?', '5만 5천 원요'],
 ['14', '이거는 천이죠?', '네 맞아요'],
 ['15', '이건 얼마예요?', '그것도 5만 5천 원요'],
 ['16', '이거 끈은 따로 없어요?', '안에 있어요'],
 ['17', '내일은 문 열어요?', '휴무입니다'],
 ['18', '며칠까지 휴무예요?', '설까지 쉬고 다음날 열 거 같아요'],
 ['19', '여기 마스크는 얼마예요?', '5천 원요'],
 ['20', '이거 나무예요? 다 돌인가요?', '나무도 있고 도자기도 있어요'],
 ['21', '이런 건 세트로 팔아요?', '네 세트로만 팔아요'],
 ['22', '이건 뭐예요?', '마블이라고 종이를 말아가지고 하는 거예요'],
 ['23', '제일 큰 거는 얼마인데요?', '세트에 7만 원이요'],
 ['24', '스

In [ ]:
faqs

In [ ]:
import jpype
from konlpy.tag import Kkma

In [ ]:
kkma = Kkma()
filter_kkma = ['NNG', 'NNP','OL','VA','VV','VXV']

def tokenizer_kkma(doc):
    # 꼬꼬마 형태소 분석기가 자바 기반이어서 파이썬에서 자바함수들을 실행할 수 있는 명령어 (jpype) 를 써줘야한다.
    jpype.attachThreadToJVM()       
    token_doc = ["/".join(word) for word in kkma.pos(doc)]
    return token_doc

def tokenize_kkma_noun_verb(doc):
    jpype.attachThreadToJVM()
    token_doc = ["/".join(word) for word in kkma.pos(doc) if word[1] in filter_kkma]
    return token_doc

In [16]:
tokenizer_kkma(faqs[0][1])

['신발/NNG',
 '은/JX',
 '여기/NP',
 '있/VV',
 '는/ETD',
 '것/NNB',
 '이/JKS',
 '다예/NNG',
 '이/VCP',
 '요/EFN',
 '?/SF']

In [ ]:
token_faqs = [(tokenizer_kkma(row[1]), row[0]) for row in faqs]
print(token_faqs[:5])
tagged_faqs = [TaggedDocument(d,[c]) for d,c in token_faqs]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: jpype._core.attachThreadToJVM is deprecated, use java.lang.Thread.attach instead
  


[(['신발/NNG', '은/JX', '여기/NP', '있/VV', '는/ETD', '것/NNB', '이/JKS', '다예/NNG', '이/VCP', '요/EFN', '?/SF'], '1'), (['230/NR', '이요/NNG'], '2'), (['네/MDN', '봄/NNG', '이/VCP', '니까/ECD', '편하/VA', '게/ECD', '신/VV', '을/ETD', '수/NNB', '있/VV', '는/ETD', '거/NNG'], '3'), (['굽/NNG', '좀/MAG', '높/VA', '은/ETD', '거/NNB', '없/VA', '나요/EFQ', '?/SF'], '4'), (['언제/MAG', '들어오/VV', '아요/EFN', '?/SF'], '5')]


In [17]:
token_faqs

[(['신발/NNG',
   '은/JX',
   '여기/NP',
   '있/VV',
   '는/ETD',
   '것/NNB',
   '이/JKS',
   '다예/NNG',
   '이/VCP',
   '요/EFN',
   '?/SF'],
  '1'),
 (['230/NR', '이요/NNG'], '2'),
 (['네/MDN',
   '봄/NNG',
   '이/VCP',
   '니까/ECD',
   '편하/VA',
   '게/ECD',
   '신/VV',
   '을/ETD',
   '수/NNB',
   '있/VV',
   '는/ETD',
   '거/NNG'],
  '3'),
 (['굽/NNG', '좀/MAG', '높/VA', '은/ETD', '거/NNB', '없/VA', '나요/EFQ', '?/SF'], '4'),
 (['언제/MAG', '들어오/VV', '아요/EFN', '?/SF'], '5'),
 (['이거/NP', '는/JX', '가죽/NNG', '이/VCP', '에요/EFN', '?/SF'], '6'),
 (['가죽/NNG', '은/JX', '얼마/NNG', '이/VCP', '에요/EFN', '?/SF'], '7'),
 (['털/NNG',
   '달리/VV',
   'ㄴ/ETD',
   '거/NNB',
   '저거/NP',
   '는/JX',
   '사이즈/NNG',
   '있/VV',
   '어요/EFN',
   '?/SF'],
  '8'),
 (['좀/MAG', '크/VA', '네/EFN', '또/MAG', '안/MAG', '들어오/VV', '아요/EFN', '?/SF'],
  '9'),
 (['가방/NNG', '매/VV', '는/ETD', '거/NNB', '보/VV', '고/ECE', '있/VV', '어요/EFN'],
  '10'),
 (['가격/NNG', '이/JKS', '얼마/NNG', '이/VCP', '에요/EFN', '?/SF'], '11'),
 (['가죽/NNG',
   '으로/JKM',
   '되/VV',
   'ㄴ/ETD',
   '거/NN

In [18]:
tagged_faqs[:5]

[TaggedDocument(words=['신발/NNG', '은/JX', '여기/NP', '있/VV', '는/ETD', '것/NNB', '이/JKS', '다예/NNG', '이/VCP', '요/EFN', '?/SF'], tags=['1']),
 TaggedDocument(words=['230/NR', '이요/NNG'], tags=['2']),
 TaggedDocument(words=['네/MDN', '봄/NNG', '이/VCP', '니까/ECD', '편하/VA', '게/ECD', '신/VV', '을/ETD', '수/NNB', '있/VV', '는/ETD', '거/NNG'], tags=['3']),
 TaggedDocument(words=['굽/NNG', '좀/MAG', '높/VA', '은/ETD', '거/NNB', '없/VA', '나요/EFQ', '?/SF'], tags=['4']),
 TaggedDocument(words=['언제/MAG', '들어오/VV', '아요/EFN', '?/SF'], tags=['5'])]

In [34]:
def run_epch(num):
  # 모델 학습
  print(num)
  d2v_faqs.train(tagged_faqs,
                 total_examples = d2v_faqs.corpus_count,
                 epochs = d2v_faqs.epochs)
  d2v_faqs.alpha -=0.0025
  d2v_faqs.min_alpha = d2v_faqs.min_alpha

In [32]:
# 모델 만들기
# cpu 몇 개 쓸 건지
import multiprocessing
# 내 컴에 있는 cpu 갯수 cores 에 저장
cores = multiprocessing.cpu_count()
# vector_size : 임베딩할 벡터 차원
# negaive : negative sampling
d2v_faqs = doc2vec.Doc2Vec(
    vector_size = 100,
    alpha = 0.025,
    min_alpha = 0.025,
    hs = 1,
    negative = 0,
    dm = 0,
    dbow_words = 1,
    min_count = 1,
    workers = cores,
    seed = 0
)

# 단어 사전 만들기
d2v_faqs.build_vocab(tagged_faqs)
run_epch

<function __main__.run_epch>

In [35]:
# 들어온 질문 형태소 분석 및 벡터화
i=0
max_correct=0
while True:
  i+=1
  run_epch(i)
  customer_arr_test = customer_arr
  store_arr_test = store_arr

  percent_of_correct = 0

  for q in range(len(customer_arr)):
      test_string = customer_arr_test[q]
      # print(test_string)
      # print(store_arr_test[q])
      token_test = tokenizer_kkma(test_string)
      predict_vector = d2v_faqs.infer_vector(token_test)
      # 1에 가까울수록 비슷한 값이고, -1에 가까울수록 상반된 값 (correlation 값)
      topn = 5

      result = d2v_faqs.docvecs.most_similar([predict_vector], topn = topn)

      for i in range(topn):
          if result[i][0] == str(q+1):
              print("**{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
              percent_of_correct+=1
          else:
              print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))
          # faqs[faqs[0]==int(result[i][0])][1]
      # print()
  if max_correct==0:
    max_correct=percent_of_correct/len(customer_arr)
  elif max_correct<percent_of_correct/len(customer_arr):
    max_correct=percent_of_correct/len(customer_arr)
  else:
    break;

1


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2위. 0.5292699337005615, 5546, 네 입어보세요
3위. 0.5201177000999451, 6767, 네, 니트 소재라 프리사이즈예요
4위. 0.5177803635597229, 4800, 네 있습니다
5위. 0.5159449577331543, 5396, 66사이즈입니다
**1위. 0.7739895582199097, 6303, 이 신발이 스니커즈로 나온 거 치곤 쿠션도 바닥도 굉장히 좋아요
2위. 0.600107729434967, 2277, 이 제품들의 경우에는 쿠션감이 이런 것보다는 되게 편해요
3위. 0.5694125890731812, 3596, 그러면 20대 남성분들에게 제일 인기 있는 이건 어떠세요?
4위. 0.5662800073623657, 1461, 사이즈 몇 신으세요?
5위. 0.5650234222412109, 1469, 한 번 신어보세요
**1위. 0.8706724643707275, 6304, 원래 8만 9천원인데 할인해서 5만 5천원이에요
2위. 0.8178989887237549, 7218, 그 세트가 29만 9천 원이에요
3위. 0.8042868971824646, 1338, 이만 팔천 원이요
4위. 0.8039772510528564, 6368, 이거는 3만원에 드려요.
5위. 0.7912005186080933, 1593, 이백사십이요
**1위. 0.8110337257385254, 6305, 지금은 매장에 있는게 다예요
2위. 0.7383599281311035, 6747, 이 제품은 가을에 나왔고, 토끼모자는 신상이에요
3위. 0.6989307999610901, 6197, 신발 이런 종류도 있어요
4위. 0.6721704006195068, 7088, 네
5위. 0.6555930376052856, 6669, 네
**1위. 0.7790423631668091, 6306, 아니요, 가격은 똑같아요
2위. 0.5775796175003052, 7145, 그 옷, 25만 원인

In [37]:
i

4

In [ ]:
input_question = input()
token_test = tokenizer_kkma(input_question)
predict_vector = d2v_faqs.infer_vector(token_test)
result = d2v_faqs.docvecs.most_similar([predict_vector],topn=5)
for i in range(5):
            print("{}위. {}, {}, {}".format(i+1, result[i][1], result[i][0], faqs[int(result[i][0])-1][2]))